In [ ]:
import torch
import torch.autograd as autograd
from torch import Tensor
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

torch.set_default_dtype(torch.float)

torch.manual_seed(1234)
np.random.seed(1234)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

In [ ]:
data = pd.read_csv('bio.csv', sep=',')
data = data.replace(np.nan, 0.)

In [ ]:
t = data['t'].to_numpy().reshape(-1,1)
cB = data['cB'].to_numpy().reshape(-1,1)
cTG = data['cTG'].to_numpy().reshape(-1,1)
cDG = data['cDG'].to_numpy().reshape(-1,1)
cMG = data['cMG'].to_numpy().reshape(-1,1)
cG = data['cG'].to_numpy().reshape(-1,1)
c = np.concatenate((cB, cTG, cDG, cMG, cG), axis=1)
c

In [ ]:
total_points = 4

t_train = np.linspace(t[0], t[-1], total_points)

idx = []
for ti in t:
    idx.append(np.where(t_train.reshape(-1,1) == ti)[0][0])
idx = np.array(idx)
idx

In [ ]:
cB_train = cB
cTG_train = cTG
cDG_train = cDG
cMG_train = cMG
cG_train = cG
cTG_train

In [ ]:
t_train = torch.from_numpy(t_train).float().to(device)
cB_train = torch.from_numpy(cB_train).float().to(device)
cTG_train = torch.from_numpy(cTG_train).float().to(device)
cDG_train = torch.from_numpy(cDG_train).float().to(device)
cMG_train = torch.from_numpy(cMG_train).float().to(device)
cG_train = torch.from_numpy(cG_train).float().to(device)
cTG_train

In [ ]:
k1 = 1.
k2 = 1.
k3 = 1.
k4 = 1.
k5 = 1.
k6 = 1.
k = [k1,k2,k3,k4,k5,k6]

layers = np.array([1,10,10,10,1])

f_hat = torch.zeros(t_train.shape[0],1).to(device)

alpha = 0.5

c_pred = torch.cat((cB_train, cTG_train, cDG_train, cMG_train, cG_train), 1)
c_pred = c_pred.to(device)
c_pred

In [ ]:
class subNN(nn.Module):
    
    def __init__(self):
        super().__init__()
        
        self.activation = nn.Tanh()
        
        self.f1 = nn.Linear(1, 10)
        self.f2 = nn.Linear(10, 10)
        self.f3 = nn.Linear(10, 10)
        self.out = nn.Linear(10, 1)
        
    def forward(self, x):
        
        if torch.is_tensor(x) != True:
            x = torch.from_numpy(x)
        
        a = x.clone().float()
        
        z_1 = self.f1(a)
        a_1 = self.activation(z_1)
        z_2 = self.f2(a_1)
        a_2 = self.activation(z_2)
        z_3 = self.f3(a_2)
        a_3 = self.activation(z_3)
        z_4 = self.out(a_3)
        a_4 = z_4.clone()
        
        return a_4


DNN Triglycerides

In [ ]:
class cTGNN():
    
    def __init__(self):
        self.dnn = subNN().to(device)
        self.loss_function = nn.MSELoss(reduction = 'mean')

        self.k1 = torch.tensor([k1], requires_grad=True).float().to(device)
        self.k2 = torch.tensor([k2], requires_grad=True).float().to(device)

        self.k1 = nn.Parameter(self.k1)
        self.k2 = nn.Parameter(self.k2)
        
        self.dnn.register_parameter('k1', self.k1)
        self.dnn.register_parameter('k2', self.k2)
        
    def pred(self, c_pred):
        self.cB = c_pred[:,0].reshape(-1,1)
        self.cDG = c_pred[:,2].reshape(-1,1)
        
    def loss(self, x, y):
        g = x.clone()
        g.requires_grad = True
        
        self.cTG = self.dnn(g)
        
        grad_cTG = autograd.grad(self.cTG, g, torch.ones(x.shape[0], 1).to(device), retain_graph=True, create_graph=True)[0]
        loss_cTG_ode = self.loss_function(grad_cTG + self.k1*self.cTG - self.k2*self.cDG*self.cB, f_hat)
        
        loss_cTG_data = self.loss_function(self.cTG, y)
        
        loss = alpha*loss_cTG_ode + (1-alpha)*loss_cTG_data
        
        return loss
    
    def closure(self):
        optimizer.zero_grad()
        
        self.pred(c_pred)
        
        loss = self.loss(t_train, cTG_train)
        
        loss.backward()
        
        return loss

cTGNN().dnn

DNN Diglycerides

In [ ]:
class cDGNN():
    
    def __init__(self):
        self.dnn = subNN().to(device)
        self.loss_function = nn.MSELoss(reduction = 'mean')

        self.k1 = torch.tensor([k1], requires_grad=True).float().to(device)
        self.k2 = torch.tensor([k2], requires_grad=True).float().to(device)
        self.k3 = torch.tensor([k3], requires_grad=True).float().to(device)
        self.k4 = torch.tensor([k4], requires_grad=True).float().to(device)

        self.k1 = nn.Parameter(self.k1)
        self.k2 = nn.Parameter(self.k2)
        self.k3 = nn.Parameter(self.k3)
        self.k4 = nn.Parameter(self.k4)
        
        self.dnn.register_parameter('k1', self.k1)
        self.dnn.register_parameter('k2', self.k2)
        self.dnn.register_parameter('k3', self.k3)
        self.dnn.register_parameter('k4', self.k4)
        
    def pred(self, c_pred):
        self.cB = c_pred[:,0].reshape(-1,1)
        self.cTG = c_pred[:,1].reshape(-1,1)
        self.cMG = c_pred[:,3].reshape(-1,1)
        
    def loss(self, x, y):
        g = x.clone()
        g.requires_grad = True
        
        self.cDG = self.dnn(g)
        
        grad_cDG = autograd.grad(self.cDG, g, torch.ones(x.shape[0], 1).to(device), retain_graph=True, create_graph=True)[0]
        loss_cDG_ode = self.loss_function(grad_cDG - self.k1*self.cTG + self.k2*self.cDG*self.cB \
                                                   + self.k3*self.cDG - self.k4*self.cMG*self.cB, f_hat)
        
        loss_cDG_data = self.loss_function(self.cDG, y)
        
        loss = alpha*loss_cDG_ode + (1-alpha)*loss_cDG_data
        
        return loss
    
    def closure(self):
        optimizer.zero_grad()
        
        self.pred(c_pred)
        
        loss = self.loss(t_train, cDG_train)
        
        loss.backward()
        
        return loss

cDGNN().dnn

DNN Monoglycerides

In [ ]:
class cMGNN():
    
    def __init__(self):
        self.dnn = subNN().to(device)
        self.loss_function = nn.MSELoss(reduction = 'mean')
        
        self.k3 = torch.tensor([k3], requires_grad=True).float().to(device)
        self.k4 = torch.tensor([k4], requires_grad=True).float().to(device)
        self.k5 = torch.tensor([k5], requires_grad=True).float().to(device)
        self.k6 = torch.tensor([k6], requires_grad=True).float().to(device)
        
        self.k3 = nn.Parameter(self.k3)
        self.k4 = nn.Parameter(self.k4)
        self.k5 = nn.Parameter(self.k5)
        self.k6 = nn.Parameter(self.k6)
        
        self.dnn.register_parameter('k3', self.k3)
        self.dnn.register_parameter('k4', self.k4)
        self.dnn.register_parameter('k5', self.k5)
        self.dnn.register_parameter('k6', self.k6)
        
    def pred(self, c_pred):
        self.cB = c_pred[:,0].reshape(-1,1)
        self.cDG = c_pred[:,2].reshape(-1,1)
        self.cG = c_pred[:,4].reshape(-1,1)
        
    def loss(self, x, y):
        g = x.clone()
        g.requires_grad = True
        
        self.cMG = self.dnn(g)
        
        grad_cMG = autograd.grad(self.cMG, g, torch.ones(x.shape[0], 1).to(device), retain_graph=True, create_graph=True)[0]
        loss_cMG_ode = self.loss_function(grad_cMG - self.k3*self.cDG + self.k4*self.cMG*self.cB \
                                                   + self.k5*self.cMG - self.k6*self.cG*self.cB, f_hat)
        
        loss_cMG_data = self.loss_function(self.cMG, y)
        
        loss = alpha*loss_cMG_ode + (1-alpha)*loss_cMG_data
        
        return loss
    
    def closure(self):
        optimizer.zero_grad()
        
        self.pred(c_pred)
        
        loss = self.loss(t_train, cMG_train)
        
        loss.backward()
        
        return loss

cMGNN().dnn

DNN Biodiesel

In [ ]:
class cBNN():
    
    def __init__(self):
        self.dnn = subNN().to(device)
        self.loss_function = nn.MSELoss(reduction = 'mean')
        
        self.k1 = torch.tensor([k1], requires_grad=True).float().to(device)
        self.k2 = torch.tensor([k2], requires_grad=True).float().to(device)
        self.k3 = torch.tensor([k3], requires_grad=True).float().to(device)
        self.k4 = torch.tensor([k4], requires_grad=True).float().to(device)
        self.k5 = torch.tensor([k5], requires_grad=True).float().to(device)
        self.k6 = torch.tensor([k6], requires_grad=True).float().to(device)
        
        self.k1 = nn.Parameter(self.k1)
        self.k2 = nn.Parameter(self.k2)
        self.k3 = nn.Parameter(self.k3)
        self.k4 = nn.Parameter(self.k4)
        self.k5 = nn.Parameter(self.k5)
        self.k6 = nn.Parameter(self.k6)
        
        self.dnn.register_parameter('k1', self.k1)
        self.dnn.register_parameter('k2', self.k2)
        self.dnn.register_parameter('k3', self.k3)
        self.dnn.register_parameter('k4', self.k4)
        self.dnn.register_parameter('k5', self.k5)
        self.dnn.register_parameter('k6', self.k6)
        
    def pred(self, c_pred):
        self.cTG = c_pred[:,1].reshape(-1,1)
        self.cDG = c_pred[:,2].reshape(-1,1)
        self.cMG = c_pred[:,3].reshape(-1,1)
        self.cG = c_pred[:,4].reshape(-1,1)
        
    def loss(self, x):
        g = x.clone()
        g.requires_grad = True
        
        self.cB = self.dnn(g)
        
        grad_cB = autograd.grad(self.cB, g, torch.ones(x.shape[0], 1).to(device), retain_graph=True, create_graph=True)[0]
        loss_cB_ode = self.loss_function(grad_cB - self.k1*self.cTG + self.k2*self.cDG*self.cB \
                                                 - self.k3*self.cDG + self.k4*self.cMG*self.cB \
                                                 - self.k5*self.cMG + self.k6*self.cG*self.cB, f_hat)
        
        return loss_cB_ode
    
    def closure(self):
        optimizer.zero_grad()
        
        self.pred(c_pred)
        
        loss = self.loss(t_train)
        
        loss.backward()
        
        return loss

cBNN().dnn

DNN Glycerol

In [ ]:
class cGNN():
    
    def __init__(self):
        self.dnn = subNN().to(device)
        self.loss_function = nn.MSELoss(reduction = 'mean')
        
        self.k5 = torch.tensor([k5], requires_grad=True).float().to(device)
        self.k6 = torch.tensor([k6], requires_grad=True).float().to(device)
        
        self.k5 = nn.Parameter(self.k5)
        self.k6 = nn.Parameter(self.k6)
        
        self.dnn.register_parameter('k5', self.k5)
        self.dnn.register_parameter('k6', self.k6)
        
    def pred(self, c_pred):
        self.cB = c_pred[:,0].reshape(-1,1)
        self.cMG = c_pred[:,3].reshape(-1,1)
        
    def loss(self, x):
        g = x.clone()
        g.requires_grad = True
        
        self.cG = self.dnn(g)
        
        grad_cG = autograd.grad(self.cG, g, torch.ones(x.shape[0], 1).to(device), retain_graph=True, create_graph=True)[0]
        loss_cG_ode = self.loss_function(grad_cG - self.k5*self.cMG + self.k6*self.cG*self.cB, f_hat)
        
        return loss_cG_ode
    
    def closure(self):
        optimizer.zero_grad()
        
        self.pred(c_pred)
        
        loss = self.loss(t_train)
        
        loss.backward(retain_graph=True)
        
        return loss

cGNN().dnn

In [ ]:
optimizers = []

PINN_cB = cBNN()
params = list(PINN_cB.dnn.parameters())
optimizer = torch.optim.Adam(params, lr=0.001)
optimizers.append(optimizer)

PINN_cTG = cTGNN()
params = list(PINN_cTG.dnn.parameters())
optimizer = torch.optim.Adam(params, lr=0.001)
optimizers.append(optimizer)

PINN_cDG = cDGNN()
params = list(PINN_cDG.dnn.parameters())
optimizer = torch.optim.Adam(params, lr=0.001)
optimizers.append(optimizer)

PINN_cMG = cMGNN()
params = list(PINN_cMG.dnn.parameters())
optimizer = torch.optim.Adam(params, lr=0.001)
optimizers.append(optimizer)

PINN_cG = cGNN()
params = list(PINN_cG.dnn.parameters())
optimizer = torch.optim.Adam(params, lr=0.001)
optimizers.append(optimizer)

In [ ]:
torch.autograd.set_detect_anomaly(True)
epoch = 0
max_epochs = 10
while epoch < max_epochs:
    # Forward
    c_pred[:,0] = PINN_cB.dnn(t_train).detach().clone().flatten()
    c_pred[:,1] = PINN_cTG.dnn(t_train).detach().clone().flatten()
    c_pred[:,2] = PINN_cDG.dnn(t_train).detach().clone().flatten()
    c_pred[:,3] = PINN_cMG.dnn(t_train).detach().clone().flatten()
    c_pred[:,4] = PINN_cG.dnn(t_train).detach().clone().flatten()
    
    # Backward
    optimizers[0].step(PINN_cB.closure)
    for p in PINN_cB.dnn.parameters():
        p.data.clamp_(min=0.)
    
    optimizers[1].step(PINN_cTG.closure)
    for p in PINN_cTG.dnn.parameters():
        p.data.clamp_(min=0.)
    
    optimizers[2].step(PINN_cDG.closure)
    for p in PINN_cDG.dnn.parameters():
        p.data.clamp_(min=0.)
    
    optimizers[3].step(PINN_cMG.closure)
    for p in PINN_cMG.dnn.parameters():
        p.data.clamp_(min=0.)
    
    optimizers[4].step(PINN_cG.closure)
    for p in PINN_cG.dnn.parameters():
        p.data.clamp_(min=0.)
    
    epoch += 1

In [ ]:
def EDO(y, prm):
    
    k1 = prm[0]
    k2 = prm[1]
    k3 = prm[2]
    k4 = prm[3]
    k5 = prm[4]
    k6 = prm[5]
    
    f = np.zeros(5)
    
    f[1] = - k1*y[1] + k2*y[2]*y[0]
    f[2] = + k1*y[1] - k2*y[2]*y[0] - k3*y[2] + k4*y[3]*y[0]
    f[3] = + k3*y[2] - k4*y[3]*y[0] - k5*y[3] + k6*y[4]*y[0]
    f[4] = + k5*y[3] - k6*y[4]*y[0]
    f[0] = + k1*y[1] - k2*y[2]*y[0] + k3*y[2] - k4*y[3]*y[0] + k5*y[3] - k6*y[4]*y[0]
    
    return f

def euler_explicite(y0, dt, tf, prm):
    
    mat_y = np.array([y0])
    
    t = np.array([0])
    while t[-1] < tf:
        y = y0 + dt * EDO(y0, prm)
        
        mat_y = np.append(mat_y, [y], axis=0)
        t = np.append(t, t[-1]+dt)
        
        y0 = np.copy(y)
    
    return t, mat_y

prm = []        
for i in range(6):
    prm.append(params[:6][i][0].cpu().detach().numpy())
    
t_euler, y_euler = euler_explicite(np.array([0,0.540121748,0.057018273,0,0]), 0.001, 6, prm)

In [ ]:
plt.plot(t_euler, y_euler[:,0])
plt.plot(t_train.detach().numpy(), PINN.dnn(t_train)[0].detach().numpy())
plt.plot(t, cB, 'o')
plt.show()

In [ ]:
plt.plot(t_euler, y_euler[:,1])
plt.plot(t_train.detach().numpy(), PINN.dnn(t_train)[1].detach().numpy())
plt.plot(t, cTG, 'o')
plt.show()

In [ ]:
plt.plot(t_euler, y_euler[:,2])
plt.plot(t_train.detach().numpy(), PINN.dnn(t_train)[2].detach().numpy())
plt.plot(t, cDG, 'o')
plt.show()

In [ ]:
plt.plot(t_euler, y_euler[:,3])
plt.plot(t_train.detach().numpy(), PINN.dnn(t_train)[3].detach().numpy())
plt.plot(t, cMG, 'o')
plt.show()

In [ ]:
plt.plot(t_euler, y_euler[:,4])
plt.plot(t_train.detach().numpy(), PINN.dnn(t_train)[4].detach().numpy())
plt.plot(t, cG, 'o')
plt.show()

In [ ]:
prm